In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot
from IPython.display import clear_output

In [2]:
import os
from PIL import Image
from matplotlib import pyplot as plt 
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def inv_sigmoid(x):
    return np.log(y/(1-y))
%matplotlib inline
path = '_data/abstract_art/Abstract_gallery/Abstract_gallery'
os.getcwd()
img_list = os.listdir(path)
def access_images(img_list,path,length):
    pixels = []
    imgs = []
    for i in range(length):
        img = Image.open(path+'\\'+img_list[i],'r')
        basewidth = 100
        img = img.resize((basewidth,basewidth), Image.ANTIALIAS)
        pix = np.array(img.getdata())
        pixels.append(pix.reshape(100,100,3))
        imgs.append(img)
    return np.array(pixels),imgs
def show_image(pix_list):
    array = np.array(pix_list.reshape(100,100,3), dtype=np.uint8)
    new_image = Image.fromarray(array)
    new_image.show()
pixels,imgs = access_images(img_list,path,1000)
pixels.shape

(1000, 100, 100, 3)

In [3]:
def define_discriminator(in_shape = (100,100,3)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [4]:
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 128 * 25 * 25
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((25, 25, 128)))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (7,7) , padding='same'))
    return model

In [5]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y
 
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y

In [7]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

In [8]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=10):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    print(dataset.shape[0])
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss, _ = d_model.train_on_batch(X, y)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
            clear_output()

In [9]:
latent_dim = 300
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
print(pixels.shape)
train(g_model, d_model, gan_model, np.array(pixels), latent_dim)
print(pixels)

[[[[239 238 236]
   [243 242 240]
   [241 241 239]
   ...
   [239 237 238]
   [239 237 238]
   [239 237 238]]

  [[238 236 234]
   [220 218 216]
   [221 219 218]
   ...
   [161 160 160]
   [175 174 175]
   [239 237 238]]

  [[225 222 220]
   [143 140 139]
   [161 159 157]
   ...
   [160 160 160]
   [176 176 177]
   [240 238 240]]

  ...

  [[247 246 244]
   [219 218 216]
   [140 139 137]
   ...
   [139 139 139]
   [161 161 161]
   [211 210 210]]

  [[248 247 245]
   [214 213 211]
   [109 108 106]
   ...
   [163 163 163]
   [169 169 169]
   [209 208 207]]

  [[243 243 241]
   [232 231 229]
   [189 189 187]
   ...
   [244 244 244]
   [240 240 240]
   [239 237 238]]]


 [[[248 248 246]
   [247 248 245]
   [249 249 247]
   ...
   [255 255 255]
   [255 255 255]
   [249 249 246]]

  [[248 248 246]
   [248 248 246]
   [237 237 235]
   ...
   [107 105 103]
   [170 168 167]
   [253 253 252]]

  [[245 245 244]
   [255 255 255]
   [146 144 139]
   ...
   [ 22  21  19]
   [100  98  96]
   [255 255

In [11]:
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
model = g_model
latent_points = generate_latent_points(300,1)
X = model.predict(latent_points)
array = np.array(X.reshape(100,100,3), dtype=np.uint8)
new_image = Image.fromarray(array)
new_image.show()
X

array([[[[174.82292 , 199.88094 , 196.02802 ],
         [206.42442 , 207.28098 , 196.26529 ],
         [199.75482 , 207.49063 , 211.5151  ],
         ...,
         [218.0264  , 200.3931  , 181.9071  ],
         [201.23134 , 187.64282 , 175.73567 ],
         [210.57559 , 202.58563 , 174.3765  ]],

        [[189.60954 , 205.35918 , 212.02261 ],
         [211.57349 , 225.63838 , 236.71117 ],
         [207.59937 , 206.53273 , 232.3247  ],
         ...,
         [209.15627 , 192.00897 , 162.32086 ],
         [206.08873 , 186.90652 , 179.11487 ],
         [204.78563 , 194.2931  , 167.63411 ]],

        [[201.33925 , 209.45268 , 219.81802 ],
         [231.40045 , 208.8106  , 226.85095 ],
         [208.28331 , 212.7565  , 232.44312 ],
         ...,
         [229.878   , 181.72208 , 182.65158 ],
         [218.38205 , 197.1462  , 177.32198 ],
         [216.1217  , 188.1749  , 166.7609  ]],

        ...,

        [[176.03423 , 116.51344 ,  74.16578 ],
         [134.41982 , 108.004585,  49.421974]